In [1]:
import pandas as pd
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA

In [2]:
path_data = "../data/"
file_ind2007 = "2007-acs-ind-codes.csv"
file_ind2010 = "2012-acs-ind-codes.csv"
df = pd.read_csv(path_data + "cps_00011_ind_geo.csv")
df_ind= pd.read_csv(path_data + "industries_acs_10_19_emp.csv")
df_occ = pd.read_csv(path_data + "occ_names_employment.csv")
# df_ind07 = pd.read_csv(path_data + file_ind2007)
# df_ind10 = pd.read_csv(path_data + file_ind2010)
n_occ = len(df_occ)
n_ind = len(df_ind)

In [3]:
ind_codes = df_ind["Code"].tolist()
occ_codes = df_occ["Code"].tolist()

In [4]:
df_ind.columns

Index(['Unnamed: 0', 'Code', 'Label', 'EMP_10_19_MEAN'], dtype='object')

In [5]:
df.columns

Index(['YEAR', 'SERIAL', 'MONTH', 'CPSID', 'ASECFLAG', 'HFLAG', 'ASECWTH',
       'MISH', 'STATEFIP', 'ASECOVERH', 'PERNUM', 'CPSIDP', 'ASECWT', 'AGE',
       'SEX', 'RACE', 'HISPAN', 'EMPSTAT', 'LABFORCE', 'OCC', 'OCC2010',
       'IND1990', 'IND', 'ASECOVERP', 'OCCLY', 'INDLY', 'IND90LY', 'OCC10LY',
       'QOCCLY', 'MIGSTA1'],
      dtype='object')

In [7]:
len(df)

4309522

In [6]:
df.head()

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,HFLAG,ASECWTH,MISH,STATEFIP,ASECOVERH,...,OCC2010,IND1990,IND,ASECOVERP,OCCLY,INDLY,IND90LY,OCC10LY,QOCCLY,MIGSTA1
0,2000,2,3,19981202520100,1,NaN,974.38,8,23,0,...,2200,850,850,0,146,850,850,2200,0,99
1,2000,2,3,19981202520100,1,NaN,974.38,8,23,0,...,9620,581,581,0,877,581,581,9620,0,99
2,2000,3,3,20000103263800,1,NaN,809.33,3,23,0,...,9999,0,0,0,0,0,0,9999,0,99
3,2000,3,3,20000103263800,1,NaN,809.33,3,23,0,...,7200,612,612,0,505,751,751,7200,0,99
4,2000,4,3,20000102304500,1,NaN,946.16,3,23,0,...,4965,651,651,0,274,651,651,4965,0,99


In [38]:
A_ind_occ = np.zeros([n_ind, n_occ])
for i, ind in enumerate(ind_codes):
    for j, occ in enumerate(occ_codes):
        A_ind_occ[i, j] += df["ASECWT"].loc[(df["IND"] == int(ind)) & \
                                    (df["EMPSTAT"].isin([10, 12])) & \
                                   (df["OCC2010"] == int(occ))].sum()
# normalize for the 10 years
A_ind_occ = A_ind_occ/10

In [40]:
np.savetxt(path_data + "asec_ind_occ_10_19_mean.csv", A_ind_occ, delimiter=",")

In [5]:
# NOTE mixing classification systems
dict_code_ind07 = dict(zip(df_ind07["2007 Census Code"], df_ind07["Industry 2007 Description"]))
dict_code_ind10 = dict(zip(df_ind10["2012 Census Code"], df_ind10["Industry 2012 Description"]))
dict_code_ind_both = copy.copy(dict_code_ind10)

In [7]:
for key07 in dict_code_ind07.keys():
    if key07 not in dict_code_ind_both.keys():
        dict_code_ind_both[key07] = dict_code_ind07[key07]